In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import natsort
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from sklearn import model_selection
import pickle
from keras.models import load_model
from sklearn.preprocessing import LabelBinarizer
from keras.callbacks import ModelCheckpoint
from keras.layers.advanced_activations import LeakyReLU

Using TensorFlow backend.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive/colab/

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
testing.40k.zip  training  Untitled0.ipynb


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import glob
path="/content/gdrive/My Drive/colab/training/training/"
path2="/content/gdrive/My Drive/colab/training/solution.csv"
im_file=os.listdir(path)
im_file=natsort.natsorted(im_file)
info=pd.read_csv(path2)

In [0]:
data=[]
img_dim=(240,240,3)
for f in im_file:
    im=cv2.imread(path+'/'+f)
    im=cv2.resize(im,(img_dim[1],img_dim[0]))
    #im=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    #cv2.imshow("img",im)
    #cv2.waitKey(0) and ord('q')
    image=img_to_array(im)
    data.append(image)
    

In [0]:
labels=[]
for i in range(5000):
    label=info.category[i]
    labels.append(label)

In [0]:
num_classes=len(np.unique(labels))
data=np.array(data,dtype="float32")/255.0
labels=np.array(labels)

In [0]:
lb=LabelBinarizer()
labels=lb.fit_transform(labels)
x_train,x_test,y_train,y_test=model_selection.train_test_split(data,labels,test_size=0.25,random_state=42)

In [0]:
aug=ImageDataGenerator(rotation_range=25,width_shift_range=0.1,height_shift_range=0.1,shear_range=0.2,horizontal_flip=True,fill_mode="nearest")

In [0]:
model=Sequential()
model.add(Conv2D(32,(3,3),padding="same",activation="linear",input_shape=(240,240,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Conv2D(32,(3,3),padding="same",activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),padding="same",activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),padding="same",activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128,(3,3),padding="same",activation="relu"))
model.add(BatchNormalization(axis=-1))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024,activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(num_classes,activation="softmax"))


In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adam(),metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 240, 240, 32)      896       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 240, 240, 32)      0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 240, 240, 32)      128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 80, 80, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 80, 80, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 80, 80, 32)        9248      
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 80, 80, 32)        0         
__________

In [0]:
checkpoint=ModelCheckpoint(filepath='/content/gdrive/My Drive/colab/model_best2.hdf5',save_best_only=True,verbose=1)
train=model.fit_generator(aug.flow(x_train,y_train,batch_size=64),validation_data=(x_test,y_test),steps_per_epoch=len(x_train)/64,epochs=150,verbose=1,callbacks=[checkpoint])

Epoch 1/150
59/58 [==============================] - 58s 975ms/step - loss: 2.9030 - acc: 0.1735 - val_loss: 2.6946 - val_acc: 0.1736

Epoch 00001: val_loss improved from inf to 2.69463, saving model to /content/gdrive/My Drive/colab/model_best2.hdf5
Epoch 2/150
59/58 [==============================] - 52s 888ms/step - loss: 2.3038 - acc: 0.2018 - val_loss: 4.2297 - val_acc: 0.1784

Epoch 00002: val_loss did not improve from 2.69463
Epoch 3/150
59/58 [==============================] - 53s 892ms/step - loss: 2.0783 - acc: 0.2416 - val_loss: 4.3409 - val_acc: 0.1648

Epoch 00003: val_loss did not improve from 2.69463
Epoch 4/150
59/58 [==============================] - 53s 892ms/step - loss: 2.0035 - acc: 0.2533 - val_loss: 2.5964 - val_acc: 0.1808

Epoch 00004: val_loss improved from 2.69463 to 2.59642, saving model to /content/gdrive/My Drive/colab/model_best2.hdf5
Epoch 5/150
59/58 [==============================] - 53s 906ms/step - loss: 1.8988 - acc: 0.2638 - val_loss: 4.2040 - val_

In [0]:
###################### Testing ##############################
path2="testing"
im_file=os.listdir(path2)
im_file=natsort.natsorted(im_file)

model=load_model("model_best2.hdf5")
lb=pickle.loads(open("lb.pickle","rb").read())

labels=[]
img_dim=(240,240,3)
for f in im_file:
    im=cv2.imread(path2+'/'+f)
    im=cv2.resize(im,(img_dim[1],img_dim[0]))
    im=im.astype("float32")/255.0
    im=img_to_array(im)
    im=np.expand_dims(im,axis=0)
    proba=model.predict(im)[0]
    idx=np.argmax(proba)
    label=lb.classes_[idx]
    labels.append(label)

Uncompressing zip file
